# 🧠 Geração de Texto com LLMs em Português
Exemplo com pipeline `text-generation` e fine-tuning mínimo com base em tópicos.


## 🤖 Geração de texto com `pipeline` (modelo GPT-2 em português)


In [ ]:
from transformers import pipeline

gerador = pipeline("text-generation", model="pierreguillou/gpt2-small-portuguese")
entrada = "Descrição de um notebook potente para programadores:"
saida = gerador(entrada, max_length=80, num_return_sequences=1)
print(saida[0]["generated_text"])

## 📚 Dataset de tópicos e textos curtos para fine-tuning

In [ ]:
from datasets import Dataset

dados = {
    "entrada": ["cadeira gamer confortável", "celular com boa câmera e bateria"],
    "texto": [
        "Cadeira gamer com apoio lombar, encosto reclinável e espuma de alta densidade.",
        "Celular com câmera de 108MP, bateria de longa duração e carregamento rápido."
    ]
}

dataset = Dataset.from_dict(dados)
dataset = dataset.train_test_split(test_size=0.5)

## 🔤 Tokenização com modelo T5 em português

In [ ]:
from transformers import AutoTokenizer

modelo = "unicamp-dl/ptt5-base-portuguese-vocab"
tokenizer = AutoTokenizer.from_pretrained(modelo)

def preprocess(example):
    input_text = ["gerar: " + x for x in example["entrada"]]
    model_inputs = tokenizer(input_text, padding="max_length", truncation=True, max_length=128)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example["texto"], padding="max_length", truncation=True, max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess, batched=True)

## 🧠 Carregar modelo e configurar treinamento


In [ ]:
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq

model = AutoModelForSeq2SeqLM.from_pretrained(modelo)

args = TrainingArguments(
    output_dir="./modelo-gerador",
    evaluation_strategy="epoch",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=10,
    predict_with_generate=True
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model)
)

trainer.train()

## ✅ Geração de novos textos com modelo ajustado

In [ ]:
entrada = "gerar: fone bluetooth resistente à água e com cancelamento de ruído"
tokens = tokenizer(entrada, return_tensors="pt")
output = model.generate(**tokens, max_new_tokens=60)
print(tokenizer.decode(output[0], skip_special_tokens=True))